In [5]:
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import os
from tqdm import tqdm
import random

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

def embed(input):
     return model(input)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [8]:
df=pd.read_csv('complete_data/2020_10_07/label_demented_within_(656)_[1266]_2020100755_dvoice_demented_within.csv') 
file_list=[]
for root, dirs, files in os.walk("./google_output"):
    for file in files:
        if file.endswith(".txt") and ('part' not in file):
            fn=os.path.join(root, file)
            file_list.append(fn)
            


In [14]:
def fn_to_id(a):
    a=a.split('/')[-1]
    a=a.replace("-", "_")
    arr=a.split('_')
    if len(arr[3])==6:
        arr[3]='20'+arr[3][4:]+arr[3][:4]
    return arr[1]+'-'+'_'.join(arr[2:4])

fn_to_id("./google_output/DVR_1-1094_20070315_773.txt")

'1-1094_20070315'

In [15]:
id_list=[]
k=0
remove_arr=[]
for i in range(len(file_list)):
    if fn_to_id(file_list[i]) in df['id_date'].unique():
        id_list.append(fn_to_id(file_list[i]))
    else:
        remove_arr.append(i)
        print(file_list[i])
        k+=1
file_list=[j for i,j in enumerate(file_list) if i not in remove_arr]
len(file_list)

./google_output/DVR_1-0532_120605_754_downsampled.txt
./google_output/DVR_3-8236_022712_811_downsampled.txt
./google_output/DVR_3-0581_081010_782_downsampled.txt
./google_output/DVR_3-0192_121710_792_downsampled.txt
./google_output/DVR_1-1507_061008_767_downsampled.txt
./google_output/DVR_1_4432_101006_767.txt
./google_output/DVR_3-0611_032511_793_downsampled.txt
./google_output/DVR_1_4432_041113_016.txt
./google_output/DVR_1-3584_062111_811_downsampled.txt
./google_output/DVR_3-7501_061410_758_downsampled.txt
./google_output/DVR_3-0224_121109_793_downsampled.txt
./google_output/DVR_0_6412_041807_755.txt
./google_output/DVR_3-8032_032911_815_downsampled.txt
./google_output/DVR_1_7254_091307_738.txt
./google_output/DVR_3-9634_110711_791_downsampled.txt
./google_output/DVR_1-2502_011607_738_downsampled.txt
./google_output/DVR_3-0022_061713_840_downsampled.txt
./google_output/DVR_3-6355_090810_791_downsampled.txt
./google_output/DVR_0_5119_072507_773.txt
./google_output/DVR_1-6792_092506_

1156

In [44]:
col_names=["id_date","diagnosis"]


for i in range(512):
    col_names.append("feature_"+str(i))
DF = pd.DataFrame(columns=col_names)          

arr=[]
skipped=0
for fn in tqdm(file_list):
    id_date = fn_to_id(fn)
    f = open(fn, "r")
    txt=(f.read())
    speaker_1_count=0
    speaker_1 = []
    speaker_2_count=0
    speaker_2 = []
    for i in txt.split("\n"):
        if "speaker 1" in i:
            speaker_1.append(i[11:])
            speaker_1_count+=len(i[11:])
        elif "speaker 2" in i:
            speaker_2.append(i[11:])
            speaker_2_count+=len(i[11:])
    if speaker_1_count<speaker_2_count:
        txt_arr=speaker_1
    else:
        txt_arr=speaker_2
    if len(txt_arr)>50:
        diagnosis= df[df["id_date"]==id_date].is_demented_at_recording.values[0]
        for i in range(30):
            txt = ' '.join(random.sample(txt_arr,25))
            feature=embed([txt]).numpy().tolist()[0]
            arr=[id_date,diagnosis]+feature
            DF=DF.append(pd.Series(arr, index=col_names), ignore_index=True)
    else:
        skipped+=1

100%|██████████| 1156/1156 [1:59:47<00:00,  6.22s/it] 


In [45]:
DF.to_pickle("./complete30x25.pkl")

In [ ]:
id_list=df_txt["file_name"].unique()
id_list[1010]='1-7367_20060522'
skipped=0
for id in tqdm(id_list):
    if (len(df_txt[df_txt["file_name"]==id].text))==30:
        diagnosis= df[df["id_date"]==id].is_demented_at_recording.values[0]
        for i in range(20):
            txt_arr=df_txt[df_txt["file_name"]==id].text.tolist()
            txt = ' '.join(random.sample(txt_arr,2))
            feature=embed([txt]).numpy().tolist()[0]
            arr=[id,diagnosis]+feature
            DF=DF.append(pd.Series(arr, index=col_names), ignore_index=True)
    else:
        skipped+=1
DF.to_pickle("./complete_sample_20.pkl")

In [ ]:
DF_tmp=DF

In [ ]:
len(DF)

In [ ]:
len(id_list)

In [ ]:
df_=pd.read_pickle("./sample_20.pkl")

In [ ]:
DF.diagnosis.sum()

In [ ]:
df_all=pd.concat([df_,DF])

In [ ]:
len(df_)

In [ ]:
df_all.to_pickle("./complete_sample_20.pkl")

In [ ]:
df_all

In [ ]:
def id_to_id(a):
    a=a.replace("-", "_")
    arr=a.split('_')
    if len(arr[2])==6:
        arr[2]='20'+arr[2][4:]+arr[2][:4]
    return arr[0]+'-'+'_'.join(arr[1:3])
df_all["id_date"] = df_all["id_date"] .apply(id_to_id)
df_all.to_pickle("./complete_sample_20.pkl")

In [ ]:
df_all